In [390]:
# Import auth client library

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [391]:
import pandas as pd
import scipy.spatial
import numpy as np
from google.cloud import bigquery
from geopy.distance import vincenty

import folium

import plotly.graph_objects as go
import plotly.express as px


# import networkx as nx
# import osmnx as ox

# Create client instance

client = bigquery.Client(project='uip-production')

In [392]:
sql = """SELECT
  distinct(dock_group_title),
  dock_group_coords.latitude,
  dock_group_coords.longitude
  
FROM
  `uip-production.bikesharing_GB_edinburghcitybikes.dockgroup_snapshots`

WHERE
  dock_group_name IN ('D68',
    'D69',
    'D70',
    'D73',
    'D79',
    'D77',
    'D83',
    'D82',
    'D80',
    'D84',
    'D87',
    'D85',
    'D86',
    'D89',
    'D91',
    'D21',
    'D94',
    'D93',
    'D96',
    'D99',
    'D100',
    'D22',
    'D105',
    'D106',
    'D109',
    'D108',
    'D110',
    'D111',
    'D112',
    'D114',
    'D115',
    'D117',
    'D118',
    'D119',
    'D121',
    'D123',
    'D124',
    'D125',
    'D127',
    'D129',
    'D130')"""
df_spen = client.query(sql).to_dataframe()
df_spen.head()

,dock_group_title,latitude,longitude
0,Constitution Street,55.975441,-3.166806
1,Marchmont Crescent,55.936432,-3.194150
2,Craigleith Road,55.956576,-3.237940
3,Abbeyhill,55.955248,-3.172216
4,Murrayfield,55.944791,-3.243673


In [393]:
sql_tot = """SELECT
  distinct(dock_group_title),
  dock_group_coords.latitude,
  dock_group_coords.longitude
  
FROM
  `uip-production.bikesharing_GB_edinburghcitybikes.dockgroup_snapshots`"""
df_tot = client.query(sql_tot).to_dataframe()
df_tot.head()

,dock_group_title,latitude,longitude
0,Borrowman Square,55.982606,-3.381455
1,Tollcross,55.944248,-3.203105
2,Canonmills,55.962804,-3.196284
3,Causewayside,55.936430,-3.180115
4,Portobello - Kings Road,55.957915,-3.118332


In [394]:
df_tot['spen'] = df_tot.dock_group_title.isin(list(df_spen.dock_group_title))
df_tot.drop_duplicates(subset='dock_group_title', inplace=True)

In [395]:
def spen(row):
  if row['spen'] == False:
    return 'No'
  else:
    return 'Yes'

In [396]:
df_tot['is_spen'] = df_tot.apply(lambda row: spen (row),axis=1)


In [397]:
m=folium.Map(location=[df_tot['latitude'].mean(),df_tot['longitude'].mean()],zoom_start=12)


In [398]:
def add_marker(row):
    if row['is_spen']=='Yes':

        marker=folium.CircleMarker([row['latitude'],row['longitude']],radius=4.5,color='#ff8000',popup=row['dock_group_title'])
        marker.add_to(m)

    else:
        marker=folium.CircleMarker([row['latitude'],row['longitude']],radius=4.5,color='blue',popup=row['dock_group_title'])
        marker.add_to(m)

In [399]:
df_tot.apply(add_marker,axis=1)
m


In [400]:
m.save('spen.html')

In [401]:
sql_trips = """SELECT trip_started_dock_group_title, vehicle_type FROM `uip-production.bikesharing_GB_edinburghcitybikes.trips` 


WHERE trip_started_dock_group_name in ('D68','D69','D70','D73','D79','D77','D83','D82','D80','D84','D87','D85','D86','D89','D91','D21','D94','D93','D96','D99','D100','D22','D105','D106','D109','D108','D110','D111','D112','D114','D115','D117','D118','D119','D121',
 'D123','D124','D125','D127','D129','D130')"""

In [402]:
df_trips = client.query(sql_trips).to_dataframe()
df_trips.head()

,trip_started_dock_group_title,vehicle_type
0,Meadow Place 2,None
1,Meadow Place 2,None
2,Meadow Place 2,None
3,Meadow Place 2,None
4,Meadow Place 2,None


In [403]:
df_trips = pd.DataFrame(df_trips.trip_started_dock_group_title.value_counts())

In [404]:
df_trips.reset_index(inplace=True)

In [405]:
df_merged = df_spen.merge(df_trips, left_on='dock_group_title',right_on='index')
df_merged.drop('index', axis=1, inplace=True)
df_merged.rename(columns={'trip_started_dock_group_title':'Hires'}, inplace=True)
df_merged.head()

,dock_group_title,latitude,longitude,Hires
0,Constitution Street,55.975441,-3.166806,889
1,Constitution Street,55.975360,-3.166442,889
2,Marchmont Crescent,55.936432,-3.194150,2587
3,Craigleith Road,55.956576,-3.237940,978
4,Abbeyhill,55.955248,-3.172216,3938


spen_m=folium.Map(location=[df_merged['latitude'].mean(),df_merged['longitude'].mean()],zoom_start=12)


for i in range(0,len(df_merged)):
   folium.Circle(
      location=[df_merged.iloc[i]['longitude'], df_merged.iloc[i]['latitude']],
      popup=df_merged.iloc[i]['dock_group_title'],
      radius=int(df_merged.iloc[i]['Hires']),
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(spen_m)


spen_m

In [406]:
map = go.Scattermapbox(
    name='Popular Spen Docking Stations',
    lon=df_merged['longitude'],
    lat=df_merged['latitude'],
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=df_merged['Hires']
    )
)

In [407]:
map

Scattermapbox({
    'lat': array([55.97544143, 55.97536038, 55.93643192, 55.95657629, 55.95524789,
                  55.94479128, 55.93796327, 55.94564842, 55.96411788, 55.93996347,
                  55.94008064, 55.93712324, 55.93650603, 55.96944275, 55.97132539,
                  55.94023763, 55.98003077, 55.97593978, 55.92758664, 55.96403105,
                  55.96345381, 55.96283984, 55.93980947, 55.95787195, 55.92781816,
                  55.93532371, 55.94485887, 55.96264211, 55.95953625, 55.93894175,
                  55.94383598, 55.94504588, 55.94191081, 55.92668428, 55.93836922,
                  55.94023763, 55.93198048, 55.94166972, 55.96076241, 55.9417913 ,
                  55.95197358, 55.95941013, 55.9397099 , 55.94442555, 55.93341636,
                  55.94894931]),
    'lon': array([-3.16680641, -3.16644163, -3.19415004, -3.23793979, -3.17221628,
                  -3.24367301, -3.18502115, -3.23184715, -3.20209475, -3.17158593,
                  -3.17174687, -3.2064

In [408]:
px.set_mapbox_access_token('pk.eyJ1IjoidG9tejg0IiwiYSI6ImNrOWdzeDl2bzBuenozbHRieXp4dmo2cmUifQ.8QfuEG6pmLxa-zdroSpbrw')

fig = px.scatter_mapbox(df_merged,title='SPEN Docking Station Popularity',
                      hover_name=df_merged['dock_group_title'],
                       lat="latitude", lon="longitude",     size='Hires',
                  color_discrete_sequence=["#0039e6"], size_max=18, zoom=11)
fig.update_layout(mapbox_style="open-street-map")

fig.show()
fig.write_html("popularity.html")


In [409]:
d = {
    'Yes': '#ff8000',
     'No':'blue'
}
colors = [d[k] for k in df_tot['is_spen'].values]

In [410]:
mapbox_access_token = 'pk.eyJ1IjoidG9tejg0IiwiYSI6ImNrOWdzeDl2bzBuenozbHRieXp4dmo2cmUifQ.8QfuEG6pmLxa-zdroSpbrw'

In [411]:
fig2 = go.Figure(go.Scattermapbox(
        lat=df_tot['latitude'],
        lon=df_tot['longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(color=colors,
            size=9
        ),
        text=df_tot['dock_group_title'],
    ))
fig2.update_layout(
    autosize=True,
    hovermode='closest',
    title='SPEN Docking Points (Orange)',
    mapbox=dict(
        style="open-street-map",
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=df_tot['latitude'].mean(),
            lon=df_tot['longitude'].mean()
        ),
       
        pitch=0,
        zoom=10
    ),
)


fig2.show()
fig2.write_html('spen_docks.html')

In [412]:
sql_spen = """
    
    
    
      SELECT 
station_id,
no_trips / months avg_monthly_trips
FROM
(
SELECT DISTINCT
trip_started_dock_group_title station_id,
COUNT(DISTINCT trip_id) no_trips,
COUNT(DISTINCT DATE_TRUNC(DATE(trip_started), MONTH)) months
FROM `uip-production.bikesharing_GB_edinburghcitybikes.trips` 
GROUP BY station_id
)
ORDER BY avg_monthly_trips DESC"""

In [413]:
df2 = client.query(sql_spen).to_dataframe()
df2

,station_id,avg_monthly_trips
0,Portobello - Kings Road,538.882353
1,Meadows East,489.366667
2,Duke Street,479.153846
3,Bristo Square,397.200000
4,St Andrew Square,371.600000
...,...,...
164,Depot,2.000000
165,City Chambers Launch Station,1.000000
166,Picady Place,1.000000
167,Cycling Scotland Conference,1.000000


In [414]:
df_final = df_spen.merge(df2, left_on='dock_group_title', right_on='station_id')
df_final.drop_duplicates(subset='dock_group_title', inplace=True)
df_final['avg_monthly_trips'] = df_final['avg_monthly_trips'].round(2)

In [420]:
fig3 = px.scatter_mapbox(df_final,title='SPEN Docking Stations Popularity - Average Monthly Hires',
                      hover_name=df_final['dock_group_title'],
                       lat="latitude", lon="longitude",     size='avg_monthly_trips',
                  color_discrete_sequence=["#ff8000"], size_max=15, zoom=11, opacity=0.8)
fig3.update_layout(mapbox_style="open-street-map")

fig3.show()
fig3.write_html("popularity_avg.html")

In [416]:
df_final.drop(['station_id', 'latitude', 'longitude'], axis=1).to_excel('popularity.xlsx')